In [1]:
import pandas as pd
import requests
import sys
from bs4 import BeautifulSoup
import multiprocessing as mp
from tqdm import tqdm
from datetime import datetime
import re
from os import listdir
from os.path import isfile, join

In [2]:
base_folder = 'Endangered Past June'
mypath = base_folder+"/past sales"
files = [mypath+'/'+f for f in listdir(mypath) if isfile(join(mypath, f))]
files.sort()


In [3]:
for file in tqdm(files):
    if '.csv' not in file:
        continue
    try:
        data = pd.read_csv(file)
    except:
        print(file)

    # loop through data
    #for index, row in tqdm(data.iterrows(), total=data.shape[0]):
    for index, row in data.iterrows():

        link = row['link']
        
        seller_link = row['seller_link']
        
        try:
            if (len(seller_link) > 0):
                continue
                
        except:

            # Detailed Scraping
            deeper_info = BeautifulSoup(requests.get(link).content, 'html.parser')

            # Get seller info
            try:
                seller_info = deeper_info.find_all('div', class_='si-content')[0]
                seller_name = seller_info.find_all('span', class_='mbg-nw')[0].text
                seller_link = seller_info.find_all('a')[0].get('href')
            except:
                seller_name = 'Unknown'
                seller_link = 'Unknown'

            # Get sales type, num bids and num sold
            try:
                #num_bids = deeper_info.find('a', id='vi-VR-bid-lnk').find_all('span')[0].text.strip()
                num_bids = deeper_info.find('div',id='mainContent').find_all('span', text=re.compile("bid"))[0].parent()[0].text.strip()
                auction_end_date = deeper_info.find('span', id='bb_tlft').text.strip()
                sell_type = 'Auction'
                num_sold = 1
            except:
                sell_type = 'Instant'
                num_bids = 0
                auction_end_date = None
                try:
                    num_sold = deeper_info.find('div',id='mainContent').find_all('a', text=re.compile("sold"))[0].text.strip().replace(' sold','')
                except:
                    num_sold = 1

            # Get locality
            try:
                origin = deeper_info.find('div',id='mainContent').find_all('div', text=re.compile("Item location"))[0].parent()[1].text.strip()
            except:
                origin = 'Unknown'

            # Description
            #description_src = deeper_info.find_all('iframe')[0].get('src')
            #description = ' '.join(BeautifulSoup(requests.get(description_src).content, 'html.parser').find('div', id='ds_div').text.strip().split())

            # save data
            data.loc[index,'auction_end_date'] = auction_end_date
            data.loc[index, 'sell_type'] = sell_type
            data.loc[index, 'num_bids'] = num_bids
            data.loc[index, 'num_sold'] = num_sold
            data.loc[index, 'origin'] = origin
            data.loc[index, 'seller_name'] = seller_name
            data.loc[index, 'seller_link'] = seller_link

    
    data.to_csv(file,index=False, encoding='utf-8')
    


100%|██████████| 230/230 [39:15<00:00, 10.24s/it]  
